In [ ]:
import ROOT
from T2KStyle import T2KStyle
c = ROOT.TCanvas('c', '', 800, 600)
cl = ROOT.TCanvas('cl', '', 1600, 1200)
cw = ROOT.TCanvas('c2', '', 1600, 600)
cw3 = ROOT.TCanvas('c3', '', 2400, 600)

In [ ]:
path = '~/LXPLUS/DESY_testbeam/tree_v3/'
f = [ROOT.TFile(path + 'mm1_360_iter0.root'),
     ROOT.TFile(path + 'mm1_370_iter0.root'),
     ROOT.TFile(path + 'mm1_380.root')]

name = ['MM1 November 360V', 'MM1 November 370V', 'MM1 November 380V']
color = [ROOT.kCyan, ROOT.kMagenta, ROOT.kYellow]

In [ ]:
# path = '~/LXPLUS/DESY_testbeam/hs_para3/'
path = '~/DATA/hs_para3/'

file = [ROOT.TFile(path + 's_200_430_02T_iter9.root'),
        ROOT.TFile(path + 's_200_480_02T_iter9.root'),
        ROOT.TFile(path + 's_412_430_02T_iter9.root'),
        ROOT.TFile(path + 's_412_480_02T_iter9.root')
       ]

title = ['200 ns 430 mm', '200 ns 480 mm', '412 ns 430 mm', '412 ns 480 mm']

In [ ]:
%jsroot on
path = '~/LXPLUS/DESY_testbeam/time_para/'
dist = '430'
file      = [ROOT.TFile(path + f'/p_116_{dist}_iter9.root'),    # 0
             ROOT.TFile(path + f'/p_200_{dist}_iter9.root'),    # 1
             ROOT.TFile(path + f'/p_412_{dist}_iter9.root'),    # 2
             ROOT.TFile(path + f'/p_505_{dist}_iter9.root'),    # 3
             ROOT.TFile(path + f'/p_610_{dist}_iter8.root')]    # 4
name = ['116 ns', '200 ns', '412 ns', '505 ns', '610 ns']
color = [ROOT.kRed, ROOT.kGreen, ROOT.kBlack, ROOT.kBlue, ROOT.kViolet, ROOT.kYellow, ROOT.kCyan, ROOT.kOrange, ROOT.kSpring, ROOT.kAzure]

In [ ]:
%jsroot on
c.cd()
# dedx = ROOT.TH1F('h', '', 100, 0., 3000.)
trunc = 0.7

hs = ROOT.THStack()
dedx = []

for file in [0, 1, 2]:
    dedx.append(ROOT.TH1F('h', '', 100, 0., 3000.))
    for entry in f[file].outtree:
        dedx_track = []
        for i in range(len(entry.charge)):
            if entry.charge[i] <= 0:
                continue
            dedx_track.append(entry.charge[i])
        dedx_track.sort()
        av = 0
        for i in range(int(trunc*len(dedx_track))):
            av += dedx_track[i] / int(trunc*len(dedx_track))
        if av != 0:
            dedx[-1].Fill(av)
    dedx[-1].SetTitle(name[file])
    dedx[-1].SetLineColor(color[file])
    dedx[-1].Fit('gaus')
    print(dedx[-1].GetFunction('gaus').GetParameter(2) / dedx[-1].GetFunction('gaus').GetParameter(1) * 100)
    hs.Add(dedx[-1])

hs.Draw('nostack')
hs.GetXaxis().SetTitle('dE/dx')
c.Draw()        
c.BuildLegend()

## Charge spreading

### 1D distro

In [ ]:
cw3.Clear()
cw3.Divide(3)

h = []
hs = []
hs.append(ROOT.THStack('hs', ''))
ls = [0, 1, 2, 3, 4]

for i in ls:
    h.append(ROOT.TH1F('h', '', 200, 0., 4000.))
    t = file[i].outtree.Project('h', f'pad_charge[][0]', f'multiplicity > 0')
    h[-1].Scale(1./h[-1].Integral())
    h[-1].SetTitle(name[i])
    h[-1].SetLineColor(color[i])
    hs[-1].Add(h[-1])
cw3.cd(1)
hs[-1].Draw('histo nostack')
hs[-1].GetXaxis().SetTitle('Q_{1}')
hs[-1].SetTitle(f'Charge in the leding')
ROOT.gPad.BuildLegend()

hs.append(ROOT.THStack('hs', ''))
for i in ls:
    h.append(ROOT.TH1F('h', '', 200, 0., 0.7))
    file[i].outtree.Project('h', f'pad_charge[][1] / pad_charge[][0]', f'multiplicity > 1')
    h[-1].Scale(1./h[-1].Integral())
    h[-1].SetTitle(name[i])
    h[-1].SetLineColor(color[i])
    hs[-1].Add(h[-1])
cw3.cd(2)
hs[-1].Draw('histo nostack')
hs[-1].GetXaxis().SetTitle('Q_{2}/Q_{1}')
hs[-1].SetTitle(f'Charge spreading')
ROOT.gPad.BuildLegend()

hs.append(ROOT.THStack('hs', ''))
for i in ls:
    h.append(ROOT.TH1F('h', '', 200, 0., 0.2))
    file[i].outtree.Project('h', f'pad_charge[][2] / pad_charge[][0]', f'multiplicity > 2')
    h[-1].Scale(1./h[-1].Integral())
    h[-1].SetTitle(name[i])
    h[-1].SetLineColor(color[i])
    hs[-1].Add(h[-1])
cw3.cd(3)
hs[-1].Draw('histo nostack')
hs[-1].GetXaxis().SetTitle('Q_{3}/Q_{1}')
hs[-1].SetTitle(f'Charge spreading')
ROOT.gPad.BuildLegend()

cw3.Draw() 

### vs the column

In [ ]:
cw3.Clear()
cw3.Divide(3)

file = [ROOT.TFile('~/LXPLUS/DESY_testbeam/test2.iter0.root')]
file_id = 0

h = []

h.append(ROOT.TH2F('h', '', 36, 0., 36., 200, 0., 4000.))
t = file[file_id].outtree.Project('h', f'pad_charge[][0]:pad_x[][]', f'multiplicity > 0')
h[-1].Scale(1./h[-1].Integral())
# h[-1].SetTitle(name[i])
hs[-1].Add(h[-1])
cw3.cd(1)
h[-1].Draw('colz')
h[-1].GetYaxis().SetTitle('Q_{1}')
h[-1].GetXaxis().SetTitle('Column')
h[-1].SetTitle(f'Charge in the leding')

h.append(ROOT.TH2F('h', '', 36, 0., 36., 200, 0., 0.7))
file[file_id].outtree.Project('h', f'pad_charge[][1] / pad_charge[][0]:pad_x[][]', f'multiplicity > 1')
h[-1].Scale(1./h[-1].Integral())
# h[-1].SetTitle(name[i])
cw3.cd(2)
h[-1].Draw('colz')
h[-1].GetYaxis().SetTitle('Q_{2}/Q_{1}')
h[-1].GetXaxis().SetTitle('Column')
h[-1].SetTitle(f'Charge spreading')

h.append(ROOT.TH2F('h', '', 36, 0., 36., 200, 0., 0.2))
file[file_id].outtree.Project('h', f'pad_charge[][2] / pad_charge[][0]:pad_x[][]', f'multiplicity > 2')
h[-1].Scale(1./h[-1].Integral())
# h[-1].SetTitle(name[i])
cw3.cd(3)
h[-1].Draw('colz')
h[-1].GetYaxis().SetTitle('Q_{3}/Q_{1}')
h[-1].GetXaxis().SetTitle('Column')
h[-1].SetTitle(f'Charge spreading')

cw3.Draw() 

In [ ]:
cw3.Clear()
cw3.Divide(3)

# file = [ROOT.TFile('~/LXPLUS/DESY_testbeam/test2.iter0.root')]
file_id = 2

h = []

h.append(ROOT.TH2F('h', '', 36, 0., 36., 200, 0., 4000.))
t = file[file_id].outtree.Project('h', f'pad_charge[][0]:pad_x[][]', f'multiplicity > 0')
h[-1].Scale(1./h[-1].Integral())
# h[-1].SetTitle(name[i])
hs[-1].Add(h[-1])
cw3.cd(1)
h[-1].Draw('colz')
h[-1].GetYaxis().SetTitle('Q_{1}')
h[-1].GetXaxis().SetTitle('Column')
h[-1].SetTitle(f'Charge in the leding')

h.append(ROOT.TH2F('h', '', 36, 0., 36., 120, -20., 100))
file[file_id].outtree.Project('h', f'time[][1] - time[][0]:pad_x[][]', f'multiplicity > 1')
h[-1].Scale(1./h[-1].Integral())
# h[-1].SetTitle(name[i])
cw3.cd(2)
h[-1].Draw('colz')
h[-1].GetYaxis().SetTitle('T_{2} - T_{1}')
h[-1].GetXaxis().SetTitle('Column')
h[-1].SetTitle(f'Charge spreading')

h.append(ROOT.TH2F('h', '', 36, 0., 36., 120, -20., 100))
file[file_id].outtree.Project('h', f'time[][2] - time[][0]:pad_x[][]', f'multiplicity > 2')
h[-1].Scale(1./h[-1].Integral())
# h[-1].SetTitle(name[i])
cw3.cd(3)
h[-1].Draw('colz')
h[-1].GetYaxis().SetTitle('T_{3} - T_{1}')
h[-1].GetXaxis().SetTitle('Column')
h[-1].SetTitle(f'Charge spreading')

cw3.Draw() 

# dE/dx

### Charge per column

In [ ]:
%jsroot on
cw3.Clear()
cw3.Divide(3)
cw3.cd(1)
file = ROOT.TFile('~/LXPLUS/DESY_testbeam/test.iter0.root')
h = ROOT.TH2F('h', '', 36, 0., 36., 200, 0., 6000.)
file.outtree.Project('h', 'charge[]:pad_x[][0]', '')
h.Draw('colz')

cw3.cd(2)
h2 = ROOT.TH2F('h2', '', 36, 0., 36., 10, 0., 10.)
file.outtree.Project('h2', 'multiplicity[]:pad_x[][0]', '')
h2.Draw('colz')

cw3.cd(3)
h3 = ROOT.TH1F('h2', '', 36, 0., 36.)
file.outtree.Project('h2', 'pad_x[][0]', '')
h3.Draw('')

cw3.Draw()